In [ ]:
# -----------------------------------------
# Task 05: SQL QA System (E-commerce Data)
# Using LangChain + SQLite + Groq (LLaMA Model)
# -----------------------------------------

In [ ]:
#Step 1: Install Packages
# ==========================
!pip install langchain langchain-community langchain-experimental groq sqlalchemy pandas


In [2]:
# Step 1: Install dependencies (run this first in terminal or Colab)
# !pip install langchain langchain-google-genai langchain-community google-generativeai sqlite3
import pandas as pd
import sqlite3
from groq import Groq
import random
from datetime import datetime, timedelta

In [ ]:
# Step 3: Initialize Groq client
# ----------------------------
api_key = "Your API"  # Replace with your actual key
client = Groq(api_key=api_key)

In [4]:
conn = sqlite3.connect("finance.db")
cursor = conn.cursor()
print("✅ Connected to database")

✅ Connected to database


In [5]:
# Create clients table
cursor.execute("""
CREATE TABLE IF NOT EXISTS clients (
    client_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    risk_profile TEXT,
    portfolio_value REAL
)
""")

# Create investments table
cursor.execute("""
CREATE TABLE IF NOT EXISTS investments (
    investment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    client_id INTEGER,
    fund_name TEXT,
    amount_invested REAL,
    date TEXT,
    FOREIGN KEY (client_id) REFERENCES clients(client_id)
)
""")

In [6]:
# ---------------------------
# Insert 30 clients
# ---------------------------
risk_profiles = ["Low", "Medium", "High"]
client_names = [f"Client {i}" for i in range(1, 31)]

clients_data = []
for i in range(30):
    name = client_names[i]
    age = random.randint(25, 65)
    risk_profile = random.choice(risk_profiles)
    portfolio_value = round(random.uniform(100000, 5000000), 2)
    clients_data.append((name, age, risk_profile, portfolio_value))

cursor.executemany("""
INSERT INTO clients (name, age, risk_profile, portfolio_value)
VALUES (?, ?, ?, ?)
""", clients_data)

# ---------------------------
# Insert 30+ investments
# ---------------------------
fund_names = ["Alpha Fund", "Beta Fund", "Gamma Fund", "Delta Fund", "Omega Fund"]
investments_data = []

for i in range(1, 31):  # client_id from 1 to 30
    for _ in range(random.randint(1, 3)):  # 1-3 investments per client
        fund_name = random.choice(fund_names)
        amount_invested = round(random.uniform(10000, 1000000), 2)
        random_days = random.randint(0, 730)  # last 2 years
        date = (datetime.now() - timedelta(days=random_days)).strftime("%Y-%m-%d")
        investments_data.append((i, fund_name, amount_invested, date))

cursor.executemany("""
INSERT INTO investments (client_id, fund_name, amount_invested, date)
VALUES (?, ?, ?, ?)
""", investments_data)

# Commit and close
conn.commit()
conn.close()

print("✅ Inserted 30 clients and 30+ investments successfully.")

✅ Inserted 30 clients and 30+ investments successfully.


In [7]:
#Step 5: Function to generate SQL queries via Groq
def generate_sql(nl_question):
    prompt = f"Generate an SQL query for SQLite based on this natural language question:\n'{nl_question}'\nReturn only the SQL query."
    
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    
    sql_query = response.choices[0].message.content.replace("```sql", "").replace("```", "").strip()
    return sql_query


In [13]:
#Step 6: Function to execute SQL & summarize results
def execute_sql(cursor, sql):
    try:
        # Clean SQL from any markdown backticks
        sql_clean = sql.replace("```sql", "").replace("```", "").strip()
        cursor.execute(sql_clean)
        rows = cursor.fetchall()
        if not rows:
            return "No results found."
        return rows  # Return raw SQL results
    except Exception as e:
        return f"⚠️ Error executing SQL: {e}"


In [9]:
questions = [
    "List clients with portfolio value over 50 lakh.",
    "Show investments in 'Alpha Fund' over 1 lakh.",
    "Which clients are high-risk and have more than 2 investments?",
    "Find the total invested amount by each client."
]


In [15]:
for idx, q in enumerate(questions, 1):
    print(f"\n{'='*50}")
    print(f"❓ Question {idx}: {q}")
    
    sql_query = generate_sql(q)
    print(f"\n📝 Generated SQL:\n{sql_query}")
    
   



❓ Question 1: List clients with portfolio value over 50 lakh.

📝 Generated SQL:
SELECT * 
FROM clients 
WHERE portfolio_value > 500000;

❓ Question 2: Show investments in 'Alpha Fund' over 1 lakh.

📝 Generated SQL:
SELECT * FROM investments 
WHERE fund_name = 'Alpha Fund' AND amount > 100000;

❓ Question 3: Which clients are high-risk and have more than 2 investments?

📝 Generated SQL:
SELECT c.client_name 
FROM clients c 
JOIN investments i ON c.client_id = i.client_id 
WHERE c.risk_level = 'high' 
GROUP BY c.client_id 
HAVING COUNT(i.investment_id) > 2;

❓ Question 4: Find the total invested amount by each client.

📝 Generated SQL:
SELECT client_id, SUM(investment_amount) AS total_invested
FROM investments
GROUP BY client_id;


In [16]:
#Save in text files
import os

# Create folder if it doesn't exist
output_folder = "query_summaries"
os.makedirs(output_folder, exist_ok=True)

# Save each summary inside the folder
for idx, q in enumerate(questions, 1):
    sql_query = generate_sql(q)
    answer = execute_sql_and_summarize(cursor, sql_query)
    
    filename = os.path.join(output_folder, f"query_{idx}_summary.txt")
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"Question: {q}\n\nSQL: {sql_query}\n\nSummary:\n{answer}")
    print(f"✅ Saved summary to {filename}")

✅ Saved summary to query_summaries\query_1_summary.txt
✅ Saved summary to query_summaries\query_2_summary.txt
✅ Saved summary to query_summaries\query_3_summary.txt
✅ Saved summary to query_summaries\query_4_summary.txt
